In [4]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel
import scipy.stats as stats
import sys

sys.path.append("../")
import vuong_tests4

In [5]:
class OLS_loglike(GenericLikelihoodModel):
    
    def __init__(self, *args,ols=False, **kwargs):
        super(OLS_loglike,self).__init__(*args,**kwargs)
        self.ols = ols

    def loglikeobs(self, params):
        y = self.endog
        x = self.exog
        mu_y = np.matmul(x,params)  
        resid = y - mu_y
        sigma = np.sqrt(np.sum(resid**2)/resid.shape[0])
        pr_y = stats.norm.logpdf( resid, loc=0,scale=sigma )
        return pr_y


def setup_shi(yn,xn,return_model=False,num_params=4):
    x1n,x2n = xn[:,0],xn[:,1:num_params+1]
    
    # model 1 grad, etc.
    model1 = sm.OLS(yn,sm.add_constant(x1n))
    model1_fit = model1.fit(disp=False)
    params1 = (model1_fit.params)
    
    model1_deriv = OLS_loglike(yn,sm.add_constant(x1n))
    ll1 = model1_deriv.loglikeobs(model1_fit.params)
    grad1 =  model1_deriv.score_obs(model1_fit.params)    
    hess1 = model1_deriv.hessian(model1_fit.params)
    
    #model 2 grad, etc.
    model2 = sm.OLS(yn,sm.add_constant(x2n))
    model2_fit = model2.fit(disp=False)
    params2 = (model2_fit.params)
    
    model2_deriv = OLS_loglike(yn,sm.add_constant(x2n))
    ll2 = model2_deriv.loglikeobs(model2_fit.params)
    grad2 =  model2_deriv.score_obs(model2_fit.params)    
    hess2 = model2_deriv.hessian(model2_fit.params)
    
    if return_model:
        return ll1,grad1,hess1,params1,model1,ll2,grad2,hess2,params2,model2
    return ll1,grad1,hess1,params1,ll2,grad2,hess2,params2

def gen_data(nobs=1000, a=0.25, num_params=4):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a*x[:,0] + a/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

yn,xn,nobs = gen_data()
ll1,grad1,hess1,params1,ll2,grad2,hess2,params2 = setup_shi(yn,xn,return_model=False,num_params=15)
print(grad1.shape,hess1.shape)

(1000, 2) (2, 2)


# a  = .25, k= 9

In [6]:
a = 0.25
num_params=9
num_sims = 10

In [7]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params,return_model=True)
gen_data_ex = lambda : gen_data(nobs=100, a=a,  num_params=num_params)
mc_out = vuong_tests4.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests4.print_mc(mc_out)
print(mc_out)

[0.8 0.  0.2] [1. 0. 0.] [1. 0. 0.]
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 1.00 & 1.00 & 0.80 & 1.00 & 1.00 & 1.00   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.00 & 0.00 & 0.20 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
llr:-1.4473112913551587, std:2.978037915823438
(array([1., 0., 0.]), array([1., 0., 0.]), array([0.8, 0. , 0.2]), array([1., 0., 0.]), array([1., 0., 0.]), array([1., 0., 0.]), -1.4473112913551587, 2.978037915823438, 4.593503995334167)


In [8]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params,return_model=True)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests4.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests4.print_mc(mc_out)
print(mc_out)

[0.3 0.  0.7] [0.8 0.  0.2] [0.8 0.  0.2]
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.80 & 0.80 & 0.30 & 0.80 & 0.80 & 1.00   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.20 & 0.20 & 0.70 & 0.20 & 0.20 & 0.00   \\
\hline
\end{tabular}
llr:-7.999848615561059, std:3.647765530210026
(array([0.8, 0. , 0.2]), array([0.8, 0. , 0.2]), array([0.3, 0. , 0.7]), array([0.8, 0. , 0.2]), array([0.8, 0. , 0.2]), array([1., 0., 0.]), -7.999848615561059, 3.647765530210026, 5.7323456390033956)


In [9]:
vuong_tests4.print_mc(mc_out)
print(mc_out)

[0.3 0.  0.7] [0.8 0.  0.2] [0.8 0.  0.2]
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.80 & 0.80 & 0.30 & 0.80 & 0.80 & 1.00   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.20 & 0.20 & 0.70 & 0.20 & 0.20 & 0.00   \\
\hline
\end{tabular}
llr:-7.999848615561059, std:3.647765530210026
(array([0.8, 0. , 0.2]), array([0.8, 0. , 0.2]), array([0.3, 0. , 0.7]), array([0.8, 0. , 0.2]), array([0.8, 0. , 0.2]), array([1., 0., 0.]), -7.999848615561059, 3.647765530210026, 5.7323456390033956)


In [10]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params,return_model=True)
gen_data_ex = lambda : gen_data(nobs=500, a=a,  num_params=num_params)
mc_out = vuong_tests4.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests4.print_mc(mc_out)
print(mc_out)

[0.8 0.  0.2] [1. 0. 0.] [1. 0. 0.]
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 1.00 & 1.00 & 0.80 & 1.00 & 1.00 & 1.00   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.00 & 0.00 & 0.20 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
llr:-4.970512571430502, std:6.035003869170188
(array([1., 0., 0.]), array([1., 0., 0.]), array([0.8, 0. , 0.2]), array([1., 0., 0.]), array([1., 0., 0.]), array([1., 0., 0.]), -4.970512571430502, 6.035003869170188, 7.575796208771791)


# a  = .25, k= 4, something not right?

In [11]:
a = 0.25
num_params=4
num_sims = 10

In [12]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params,return_model=True)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests4.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests4.print_mc(mc_out)
print(mc_out)

[1. 0. 0.] [1. 0. 0.] [1. 0. 0.]
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 1.00 & 1.00 & 1.00 & 1.00 & 1.00 & 1.00   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
llr:-2.7553140644736547, std:4.400872514133004
(array([1., 0., 0.]), array([1., 0., 0.]), array([1., 0., 0.]), array([1., 0., 0.]), array([1., 0., 0.]), array([1., 0., 0.]), -2.7553140644736547, 4.400872514133004, 5.478577368482261)


In [13]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params,return_model=True)
gen_data_ex = lambda : gen_data(nobs=500, a=a,  num_params=num_params)
mc_out = vuong_tests4.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests4.print_mc(mc_out)
print(mc_out)

[0.9 0.  0.1] [0.9 0.  0.1] [0.9 0.  0.1]
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.90 & 0.90 & 0.90 & 0.90 & 0.90 & 1.00   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.10 & 0.10 & 0.10 & 0.10 & 0.10 & 0.00   \\
\hline
\end{tabular}
llr:-0.7433703350525509, std:8.382010428020116
(array([0.9, 0. , 0.1]), array([0.9, 0. , 0.1]), array([0.9, 0. , 0.1]), array([0.9, 0. , 0.1]), array([0.9, 0. , 0.1]), array([1., 0., 0.]), -0.7433703350525509, 8.382010428020116, 7.82719672272187)


# a  = .25, K = 19

In [14]:
a = 0.25
num_params=19
num_sims = 10

In [15]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params,return_model=True)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests4.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests4.print_mc(mc_out)
print(mc_out)

[0.1 0.  0.9] [0.6 0.  0.4] [0.6 0.  0.4]
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.70 & 0.90 & 0.10 & 0.60 & 0.60 & 1.00   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.30 & 0.10 & 0.90 & 0.40 & 0.40 & 0.00   \\
\hline
\end{tabular}
llr:-11.150527728882029, std:4.668205111611038
(array([0.7, 0. , 0.3]), array([0.9, 0. , 0.1]), array([0.1, 0. , 0.9]), array([0.6, 0. , 0.4]), array([0.6, 0. , 0.4]), array([1., 0., 0.]), -11.150527728882029, 4.668205111611038, 6.813275412396527)


In [16]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params,return_model=True)
gen_data_ex = lambda : gen_data(nobs=500, a=a,  num_params=num_params)
mc_out = vuong_tests4.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests4.print_mc(mc_out)
print(mc_out)

[0.4 0.  0.6] [0.9 0.  0.1] [0.9 0.  0.1]
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.80 & 0.80 & 0.40 & 0.90 & 0.90 & 1.00   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.20 & 0.20 & 0.60 & 0.10 & 0.10 & 0.00   \\
\hline
\end{tabular}
llr:-9.66395540863898, std:6.230254117164672
(array([0.8, 0. , 0.2]), array([0.8, 0. , 0.2]), array([0.4, 0. , 0.6]), array([0.9, 0. , 0.1]), array([0.9, 0. , 0.1]), array([1., 0., 0.]), -9.66395540863898, 6.230254117164672, 8.432150918700124)


# evidence of power

In [17]:
a1,a2 = np.sqrt(1.09-1), 0.00
num_params= 9
num_sims = 10

def gen_data2(nobs=1000, a1=np.sqrt(1.09-1), a2=0.00 , num_params=19):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a1*x[:,0] + a2/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

In [18]:
a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params,return_model=True)
gen_data_ex = lambda : gen_data2(nobs=250, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests4.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests4.print_mc(mc_out)
print(mc_out)

[0.9 0.1 0. ] [0.5 0.5 0. ] [0.5 0.5 0. ]
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.70 & 0.70 & 0.90 & 0.50 & 0.50 & 0.50   \\
Model 1 & 0.30 & 0.30 & 0.10 & 0.50 & 0.50 & 0.50   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
llr:8.366746349811006, std:6.808144685226765
(array([0.7, 0.3, 0. ]), array([0.7, 0.3, 0. ]), array([0.9, 0.1, 0. ]), array([0.5, 0.5, 0. ]), array([0.5, 0.5, 0. ]), array([0.5, 0.5, 0. ]), 8.366746349811006, 6.808144685226765, 5.910660747282486)


In [19]:
a1,a2 = np.sqrt(1.09**.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params,return_model=True)
gen_data_ex = lambda : gen_data2(nobs=500, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests4.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests4.print_mc(mc_out)
print(mc_out)

[1. 0. 0.] [0.6 0.4 0. ] [0.6 0.4 0. ]
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.70 & 0.70 & 1.00 & 0.60 & 0.60 & 0.50   \\
Model 1 & 0.30 & 0.30 & 0.00 & 0.40 & 0.40 & 0.50   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
llr:7.9448249682094625, std:6.066717567257404
(array([0.7, 0.3, 0. ]), array([0.7, 0.3, 0. ]), array([1., 0., 0.]), array([0.6, 0.4, 0. ]), array([0.6, 0.4, 0. ]), array([0.5, 0.5, 0. ]), 7.9448249682094625, 6.066717567257404, 5.6962677338626655)


In [20]:
a1,a2 = 0.00,np.sqrt(1.09-1)
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params,return_model=True)
gen_data_ex = lambda : gen_data2(nobs=250, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests4.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests4.print_mc(mc_out)
print(mc_out)

[0. 0. 1.] [0.1 0.  0.9] [0.1 0.  0.9]
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.10 & 0.10 & 0.00 & 0.10 & 0.10 & 0.50   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.90 & 0.90 & 1.00 & 0.90 & 0.90 & 0.50   \\
\hline
\end{tabular}
llr:-18.53550919837079, std:6.235404924750666
(array([0.1, 0. , 0.9]), array([0.1, 0. , 0.9]), array([0., 0., 1.]), array([0.1, 0. , 0.9]), array([0.1, 0. , 0.9]), array([0.5, 0. , 0.5]), -18.53550919837079, 6.235404924750666, 5.994634916285026)


In [21]:
a1,a2 =  0.00,np.sqrt(1.09**.5-1)
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params,return_model=True)
gen_data_ex = lambda : gen_data2(nobs=500, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests4.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests4.print_mc(mc_out)
print(mc_out)

[0. 0. 1.] [0.1 0.  0.9] [0.1 0.  0.9]
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.20 & 0.30 & 0.00 & 0.10 & 0.10 & 0.70   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.80 & 0.70 & 1.00 & 0.90 & 0.90 & 0.30   \\
\hline
\end{tabular}
llr:-13.610301936900337, std:4.971498593379119
(array([0.2, 0. , 0.8]), array([0.3, 0. , 0.7]), array([0., 0., 1.]), array([0.1, 0. , 0.9]), array([0.1, 0. , 0.9]), array([0.7, 0. , 0.3]), -13.610301936900337, 4.971498593379119, 5.349105149475334)
